## Learning Objectives:

In this tutorial, we will learn to implement the following:

> Section-1
* Building a neural network from scratch with tensorflow operations

> Section-2
* Keras Sequential API
* Keras Functional API
* Keras Model subclassing
* Callbacks

# **Section - 1**

## Introduction to Keras

Keras is an open-source neural network library written in Python that allows you to build and train deep learning models. It provides a user-friendly and modular API for creating and configuring deep neural networks with high-level abstractions. Keras is built on top of other popular deep learning frameworks such as TensorFlow, Theano, and CNTK. It has a wide range of applications in areas such as computer vision, natural language processing, and time-series forecasting.

### Setup Steps:

## Import libraries

In [ ]:
# Import TensorFlow and Keras (a high-level API within TensorFlow)
import tensorflow as tf  # Core TensorFlow library
from tensorflow import keras  # Keras module within TensorFlow for building models

# Import additional libraries for mathematical operations and data handling
import math  # Provides mathematical functions (like sqrt, sin, etc.)
import numpy as np  # For numerical computing, including arrays and matrix operations
import pandas as pd  # For handling and analyzing structured data (e.g., DataFrames)
import matplotlib.pyplot as plt  # For plotting and visualization of data

# Import specific Keras layers and tools for building neural networks
from keras.layers import Dense, Flatten  # Dense: fully connected layer, Flatten: reshapes input data
from keras import Input  # Used to define the input layer of a model
from tensorflow.keras.utils import plot_model  # To visualize the architecture of a neural network

# Import the MNIST dataset (handwritten digit dataset) for training and testing
from tensorflow.keras.datasets import mnist  # Provides training and test data for the MNIST dataset

## Basic Sequential Model

We want to build a sequential model. This means that the layers of our neural network are stacked sequentially. The approach is as follows:
1.  First implement a class to build a dense layer. We call it "NaiveDense"
2.  Implement a class ("NaiveSequential") to stack the layers sequentially and build a sequential model.


In [ ]:
# Implementing a custom dense (fully connected) layer class
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        """
        Initializes the layer with the given input size, output size, and activation function.
        Args:
            input_size (int): Number of input features.
            output_size (int): Number of neurons in this layer.
            activation (function): Activation function (e.g., ReLU, Sigmoid) applied to the output.
        """
        self.activation = activation  # Store the activation function for later use

        # Define the shape of the weights matrix (input_size x output_size)
        w_shape = (input_size, output_size)

        # Initialize the weights with small random values (between 0 and 0.1)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        # Store the weights as a TensorFlow variable to allow for updates during training
        self.w = tf.Variable(w_initial_value)

        # Define the shape of the bias vector (one bias per output neuron)
        b_shape = (output_size,)

        # Initialize the biases to zero
        b_initial_value = tf.zeros(b_shape)
        # Store the biases as a TensorFlow variable to enable updates during training
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        """
        This method makes the class instance callable like a function.
        It computes the layer's output by performing a matrix multiplication and adding the bias.
        Args:
            inputs (Tensor): Input data (batch of features).
        Returns:
            Tensor: Activated output after applying the activation function.
        """
        # Perform matrix multiplication between inputs and weights, add bias, and apply activation
        return self.activation(tf.matmul(inputs, self.w) + self.b)  # Bias is broadcasted to match dimensions

    @property
    def weights(self):
        """
        A property to access the weights and biases of the layer.
        Returns:
            tuple: (weights, biases)
        """
        return (self.w, self.b)  # Returns weights and biases as a tuple

We implemented a dense layer. Now we will stack them together sequentially in our NaiveSequential class

In [ ]:
class NaiveSequential:
    def __init__(self,layers):       # Layers: list of layer objects
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:    # Ouptut of the prev layer is the input to the next layer
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:     # Save weights of each layer to a list
            weights += layer.weights  # Q: What does layer.weights return?
        return weights                # A: layer.weights calls the function layer.weights() since it decorated with @property. It returns (w,b)

Sequential stacking of dense layer is implemented.

Further, instantiate NaiveSequential class and make our first NN model.

In [ ]:
# Define the model
model = NaiveSequential([
        NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
        NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
# Q: What input argument does NaiveSequential take? A: list of layer objects
# Q: What is the input and output dimension of the overall model? A: input dim = 784, output dim = 10
# Q: Can the output_size of 1 layer be different from the input_size of the next layer? A: No, they have to be the same.

'model' is the object of 'NaiveSequential' class. This class has 'weights' as one of the methods of the class which is accessed using 'model.weights'. When 'NaiveDense' class is used as a function it will return the initial values of weights and biases.

In [ ]:
print(model.weights)

The sequential model is untrained and currently not useful.

We must train the model to make it learn useful representaions but first we need data.

So, we will solve a calssification problem by using above sequential model using [MNIST dataset](https://keras.io/api/datasets/mnist/).


### MNIST Dataset

The **MNIST dataset** contains images of handwritten digits. It has a training set of 60,000 images, and a test set of 10,000 images.

<center><img src='https://storage.googleapis.com/tfds-data/visualization/fig/mnist-3.0.1.png' width=400px></center>

Now, let us do the following:
1. Load the data
2. Reshape the data according to the input shape of the model
3. Normalize the data

In [ ]:
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# The MNIST dataset contains 28x28 grayscale images of handwritten digits (0-9).

# Display the shape of the training images and labels
print(f"train_images.shape = {train_images.shape}")  # Output: (60000, 28, 28)
print(f"train_labels.shape = {train_labels.shape}")  # Output: (60000,)

# Reshape and normalize the data for use in neural networks
train_images = train_images.reshape((len(train_images), 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((len(test_images), 28 * 28)).astype("float32") / 255

# Explanation:
# - Reshape the images from (60000, 28, 28) to (60000, 784) to flatten them into 1D vectors.
# - Convert pixel values from integers (0 to 255) to floats (0.0 to 1.0) by dividing by 255.
# - Normalization ensures that the input values are scaled, improving model convergence.

# Display the training labels
print(train_labels)
# Output: [5 0 4 ... 5 6 8]
# The labels represent the digits (0 to 9) corresponding to each image.

### Visualize Image

In [ ]:
# Read image
img =train_images[2].reshape(28,28)
plt.imshow(img, cmap="gray")
plt.grid(True)
plt.colorbar()

We divide the data into batches. For this operation, we implement a class for Batch Generation.

In [ ]:
# A class to generate batches of data for training or testing
class BatchGenerator:

    def __init__(self, images, labels, batch_size=128):
        # Ensure the number of images and labels are the same
        assert len(images) == len(labels), "Images and labels must have the same length"

        # Initialize starting index for batch generation
        self.index = 0

        # Store the provided images and labels
        self.images = images
        self.labels = labels

        # Set the batch size (default: 128 samples per batch)
        self.batch_size = batch_size

        # Calculate the total number of batches needed (rounding up if necessary)
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        """Fetch the next batch of images and labels."""
        # Get the current batch of images and labels based on the current index
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]

        # Increment the index to point to the next batch
        self.index += self.batch_size

        # Return the batch of images and labels
        return images, labels


`batch_generator.num_batches` is an attribute of the 'BatchGenerator' class which represents the total number of batches that can be generated from the given dataset of images and labels, based on the specified batch size. It is calculated as the total number of images divided by the batch size, rounded up to the nearest integer using the `math.ceil()` function.

In [ ]:
batch_generator = BatchGenerator(train_images, train_labels)  # Initialize the batch generator
batch_generator.num_batches  # Access the number of batches

`batch_generator.next()` is a method of the BatchGenerator class which generates the next batch of images and labels from the dataset. Each time next() is called, it returns a tuple of images and labels corresponding to the next batch of size batch_size, and updates the internal index pointer to point to the start of the next batch.

In [ ]:
batch_generator.next()

Once we have defined the model all we have to do for train the model is:

1.   model.compile()
2.   model.fit()

We should know what goes on behind the scenes. The steps involved in training a model are as follows:

*Training steps:*

1. Compute the predictions using current weights (Forward Pass).
2. Compute the loss value for these predictions.
3. Compute the gradient with regard to model weights.
4. Update the weights.

In [ ]:
# One_training_step() function gives the idea of how loss is computed and
# Layer parameters (weights and biases) are updated

def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:                   # GradientTape() is the computational graph
        predictions = model(images_batch)               # Forward pass.
        per_sample_losses = keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)       # Define loss

        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)      # Compute gradients
    update_weights(gradients, model.weights)                    # Update the weights
    return average_loss

learning_rate = 1e-3
def update_weights(gradients, weights):
    for g,w in zip(gradients, weights):
        w.assign_sub(g*learning_rate)             # w -= g*lr ; w = w - lr*g

loss_observed = []

# Full training loop
def fit(model, images, labels, epochs, batch_size=128):
    for epoch in range(epochs):                       # Repeat for epochs
        print(f"Epoch {epoch+1}/{epochs}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):      # Go through all mini-batches in the data
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter%100 == 0:
                print(f"    loss at batch {batch_counter:<3} : {loss:.2f}")
        loss_observed.append(loss)

Train the model on MNIST data set.

In [ ]:
fit(model, train_images, train_labels, epochs=10, batch_size=128)

# Q: We didn't do a compile step.... or did we?
# A: In model.compile(), we pass information about the loss function, optimizer, and evaluation metric.
#     In our naive implementation, instead of defining a separate compile() function, we have defined the loss
#     inside one_training_step; implemented the optimizer 'mini-batch gradient descent' in update_weights();
#     and we are doing the evaluation (accuracy) separately in a later cell.

In [ ]:
# Plot the loss observed during training

plt.plot(range(1,11), loss_observed, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

After 10 epochs the loss has come down.

The model has definitely learned something. Lets evaluate how accurately it can predict labels for images **it has not seen before**. These are the **images in the test set**.

Use the "accuracy" metric. Here we simply find the fraction of times the model succeeded in predicting the correct label.

When using Keras, we would mention this metric in `model.compile()`. (More on Keras later)


In [ ]:
#  Evaluation step
predictions = model(test_images).numpy()                # Gives probabilities of class labels
predicted_labels = np.argmax(predictions, axis=1)       # Selecting class label which has highest probability value

matches = predicted_labels == test_labels               # Check how many are correctly predicted

print(f"accuracy: {matches.mean():.2f}")
print(predictions.shape)
print(predicted_labels)
print(matches)

## Section - 2

---



## **Different APIs**
1. Sequential Model
2. Functional API
3. Model subclassing



---



---



### Sequential API

Whatever has been implemented so far can be done alternatively using the Sequential class in keras. In the following approach, layers are passed as a list.

Defining the same old model by subclassing the Model class [[Reference](https://keras.io/api/models/model/)].

In [ ]:
# from keras.layers import Dense, Flatten
# from keras import Input

seq_model = keras.Sequential([
                         Dense(64, activation="relu"),
                         Dense(10, activation="softmax")
])
# Q: Do you notice a difference in arguments of the Dense layers, compared to our implementation?
# A: We did not have write input_shape explicitly for each layer. It automatically inferred by Keras

Alternatively, instead of passing layers as list, we can build a sequential model by adding layers incrementally to the model.

In [ ]:
# Initialize a Sequential model - a linear stack of layers
seq_model_inc = keras.Sequential()

# Add a dense layer with 64 neurons and ReLU activation function
# This hidden layer learns patterns from the input data
seq_model_inc.add(Dense(64, activation="relu"))

# Add a dense output layer with 10 neurons and softmax activation function
# This layer will classify the input data into one of 10 classes (for multi-class classification)
seq_model_inc.add(Dense(10, activation="softmax"))

Notice that we have not yet provided information of input dimensions.

These layers are referred to as symbolic layers.

The model's layer weights are not created until the model is built.

In [ ]:
try:
    seq_model_inc.weights
except:
    print("seq_model_inc.weights did not work because model was not built and weights were not initialized.")

To create a weights you need to call on some data or call its build method with input shape

In [ ]:
seq_model_inc.build(input_shape=(None, 3))  # None means it can take any batch size; 3 is the number of features in your input
seq_model.build(input_shape=(None, 3))

# seq_model_inc.weights

In [ ]:
seq_model.summary()

Q: Verify the number of parameters by a quick calculation?

A: 650 = 10*64 + 10

weight matrix has 64*10 weights and 10 biases for the 10 neurons

**Specifying input shape in advance**

In [ ]:
model_seq = keras.Sequential(name="sequential_model")
model_seq.add(keras.Input(shape=(3, )))   #specifying the input here
model_seq.add(keras.layers.Dense(64, activation=tf.nn.relu, name="first_layer"))
model_seq.add(keras.layers.Dense(10, activation=tf.nn.softmax, name="second_layer"))

In [ ]:
model_seq.summary()

### 2. Functional API

We will use the Keras functional API to create the same model. Keras functional API can create more flexible models than Sequential API. It can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

Key Idea- Expresses each layer as a function of the previous layer.

<center>(input: 3-dimensional vectors)</center>
<center>  ↧ </center>
<center>[Dense (64 units, relu activation)]</center>
<center>   ↧ </center>
<center>(output: 10 units, softmax activation)</center>

Defining the same old model by subclassing the Model class [[Reference](https://keras.io/api/models/model/)].

In [ ]:
inputs = Input(shape=(3,), name="input_layer")
features = Dense(64, activation="relu",name="first_layer")(inputs)          #f(inputs)
outputs = Dense(10, activation="softmax", name="output_layer")(features)    #f(features)
fun_model = keras.Model(inputs, outputs)

In [ ]:
fun_model.summary()

We get effectively the same summary becuase we have implemented the same model using the functional API.

In [ ]:
plot_model(fun_model, show_shapes=True)

A deeper network.

In [ ]:
# from keras.layers import Dense
# import keras
# node = Layer(nodes, extra_params)(prev_node)
inputs = keras.Input(shape=(64,))
dense1 = Dense(32, activation='relu')(inputs)
dense2 = Dense(32, activation='relu')(dense1) # Defining dense2 node whose parent is dense1
outputs = Dense(4, activation='softmax')(dense2) # Defining output node where parent is dense2
model = keras.Model(inputs=inputs, outputs=outputs, name="linear_topology")

In [ ]:
model.summary()

In [ ]:
plot_model(model)

An example where the Sequential API would not be sufficeint.

**Multi-Input and Multi-output:** Consider an example of building a system to rank customer tickets by priority and route them to the appropriate departments.

Outputs: model need to give two outputs
1. First task of the model is to classify the tickets into priority and non priority (Binary classification)

2. Second task is to route the ticket to appropriate department (Multi-class classification based on the number of departments)

These two task are to be done simultaneously

Inputs:
1. Title of the ticket (text input)
2. The text body of the ticket (text input)
3. Any tags added by the user

Q. Is it possible to build the model sequentially?

A: No, we cannot build a multi-input , multi-output model through the sequential API, because, by definition itself, the required model is not sequential.

In [ ]:
# Define the size of the input features and the number of classes for outputs
vocabulary_size = 10000   # Number of unique words/tokens in the vocabulary
num_tags = 100            # Number of unique tags
num_departments = 4       # Number of departments (classification output)

# Inputs
title = keras.Input(shape=(vocabulary_size,), name="title")  # Input for the title with a shape based on vocabulary size
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")  # Input for the text body
tags = keras.Input(shape=(num_tags,), name="tags")  # Input for the tags (binary vector with num_tags size)

# Concatenate the inputs into a single layer to combine the features
features = keras.layers.Concatenate()([title, text_body, tags])  # Merging title, text body, and tags into one feature layer
features = keras.layers.Dense(64, activation="relu")(features)   # Applying a Dense layer with 64 units and ReLU activation

# Outputs
priority = keras.layers.Dense(1, activation="sigmoid", name="priority")(features)  # Output layer for binary classification (priority)

department = keras.layers.Dense(num_departments, activation="softmax", name="department")(features)  # Output layer for department classification (softmax for multi-class)

# Define the model with multiple inputs and multiple outputs
model = keras.Model(inputs=[title, text_body, tags],
                    outputs=[priority, department])  # Model with inputs as title, text body, and tags; outputs as priority and department


In [ ]:
model.summary()

In [ ]:
plot_model(model)

Reusing the model by training intermediate layer output

In [ ]:
# Extract the output of the 4th layer of the model (the features from a previous layer in the model)
features = model.layers[4].output  # The output from the 4th layer of the model, which will be used as input for the next layer

# Add a Dense layer to predict 'difficulty' with 3 possible classes (softmax for multi-class classification)
difficulty = keras.layers.Dense(3, activation="softmax", name="difficulty")(features)  # A Dense layer with 3 units and softmax activation for difficulty classification

In [ ]:
# Create a new model by specifying the inputs and outputs

new_model = keras.Model(inputs=[title, text_body, tags],      # The inputs for the new model: title, text_body, and tags
                        outputs=[priority, department, difficulty])  # The outputs for the new model: priority, department, and difficulty

In [ ]:
keras.utils.plot_model(new_model)

### 3. Subclassing the Model class

We saw how the functional API enabled us to make more complex models compared to the sequential API. We moved up the ladder of progressive disclosure of complexity.


Defining the same old model by subclassing the Model class [[Reference](https://keras.io/api/models/model/)].


In [ ]:
class CustomerTicketModel(keras.Model):

# Define the layers in the __init__ method
    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = keras.layers.Concatenate()
        self.mixing_layer = keras.layers.Dense(64, activation="relu")
        self.priority_scorer = keras.layers.Dense(1, activation="sigmoid")
        self.department_classifier = keras.layers.Dense(num_departments, activation="softmax")


# Define the relationship between layers in the call method
# See Section 7.2.3 in Francois chollet for more details
# You implement custom layers by writing a call method.
    def call(self, inputs):
        # input should be dictionary type
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)

        priority = self.priority_scorer(features)
        department = self.department_classifier(features)

        return priority, department

In [ ]:
sub_class_model = CustomerTicketModel(num_departments=4)

In [ ]:
try:
    sub_class_model.summary()
except:
    print("summary() did not work because we have not built the model")

In [ ]:
# here model is built by calling the data since build() method is not
# defined in model subclass
# generate random data
title_data = np.random.randint(0, 2, size=(1000,vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(1000,vocabulary_size))
tags_data = np.random.randint(0, 2, size=(1000,num_tags))

priority, department = sub_class_model({"title":title_data,
                                        "text_body":text_body_data,
                                        "tags":tags_data})

In [ ]:
sub_class_model.summary()

## Building the model using custom dense layer and functional API

Building custom layer [[Reference](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer)].

In [ ]:
from keras.initializers import RandomNormal

In [ ]:
class Custom_Dense(keras.layers.Layer):
    def __init__(self, units, activation=None):
        super() .__init__()
        self.units = units
        self.activation = activation

    # Subclassing gives us the flexibility here to initialize weights on our own
    def build(self, input_shape):
        input_dim = input_shape[-1]
        std_dev = np.sqrt(2/(input_dim + self.units))
        self.W = self.add_weight(shape=(input_dim, self.units),
                                initializer=RandomNormal(stddev=std_dev))
        self.b = self.add_weight(shape=(self.units,),
                                    initializer="zeros")

    def call(self, inputs):
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y


We can even define custom metrics and custom loss functions using the subclassing API. Refer to Section 7.3.1 of Chollet for details.

### Using custom dense layer with functional API


In [ ]:
inputs = Input(shape=(28*28,))
features = Custom_Dense(512, activation=tf.nn.relu)(inputs)
features = Custom_Dense(128, activation=tf.nn.relu)(features)
outputs = Custom_Dense(10, activation=tf.nn.softmax)(features)

model = keras.Model(inputs,outputs)

In [ ]:
model.summary()

In [ ]:
plot_model(model)

In [ ]:
model.compile(optimizer =keras.optimizers.RMSprop(),
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ["accuracy"])

In [ ]:
train_x = train_images[10000:]
train_y = train_labels[10000:]
val_x = train_images[:10000]
val_y = train_labels[:10000]

In [ ]:
history = model.fit(x=train_x, y=train_y, epochs=10,
                    validation_data=(val_x, val_y))

In [ ]:
data = pd.DataFrame(history.history)
data.head()

In [ ]:
plt.plot(range(1,11),data['loss'], label="Training Loss")
plt.plot(range(1,11),data['val_loss'],label="validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

Q. What is the difference between evaluate() and predict()

A: evaluate() returns the loss score and evaluation score. predict() runs a forward pass for the given input data.

"predict" is used to make predictions on new data using a trained model. Given an input tensor, the "predict" function outputs the corresponding predictions generated by the model.

"evaluate" is used to evaluate the performance of a trained model on a given dataset. Given an input dataset, the "evaluate" function computes the model's performance metrics, such as accuracy, loss, or any other metrics defined during model compilation.

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
class_predicted = np.argmax(model.predict(test_images), axis=1)
accuracy = np.sum(class_predicted == test_labels)/len(test_labels)
print(accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(classification_report(test_labels, class_predicted))

In [ ]:
pd.Series(test_labels).value_counts()

In [ ]:
# Given values for label 0
TP = 974  # True Positives
FP = 15   # False Positives
FN = 6    # False Negatives

# Calculating precision, recall, and F1 score
precision = TP / (TP + FP)  # Precision calculation
recall = TP / (TP + FN)     # Recall calculation
f1 = 2 * precision * recall / (precision + recall)  # F1 score calculation

precision, recall, f1  # Output the precision, recall, and F1 score

In [ ]:
print(confusion_matrix(test_labels, class_predicted))

In [ ]:
# Save model

## Using Callbacks

A callback is an object that can perform actions at various stages of training (e.g. at the start or end of an epoch, before or after a single batch, etc).

You can use callbacks to:

* Write TensorBoard logs after every batch of training to monitor your metrics
* Periodically save your model to disk
* Do early stopping
* Get a view on internal states and statistics of a model during training

Access Keras callbacks [here](https://keras.io/api/callbacks/)

In [ ]:
# Build model using functional API
inputs = Input(shape=(28*28,))
features = Dense(512,activation="relu")(inputs)
features = keras.layers.Dropout(0.5)(features)
outputs = Dense(10,activation="softmax")(features)

mnist_model = keras.Model(inputs, outputs)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [ ]:
callbacks_list = [EarlyStopping(monitor="val_loss", patience=2),
                  # Added .keras extension to the filepath for saving the entire model
                  ModelCheckpoint("mnist_model_checkpoint.keras", save_best_only=True),
                  TensorBoard(log_dir="./tensorboard_files")]

In [ ]:
mnist_model.compile(optimizer =keras.optimizers.Adam(),
                    loss = keras.losses.SparseCategoricalCrossentropy(),
                    metrics = ["accuracy"])

In [ ]:
mnist_model.fit(x= train_x, y= train_y,
                epochs= 5,
                validation_data= (val_x, val_y),
                callbacks= callbacks_list,)

###TensorBoard

It is a visualization tool provided with TensorFlow.

This callback logs events for TensorBoard, including:

* Metrics summary plots
* Training graph visualization
* Weight histograms
* Sampled profiling

In [ ]:
!ls

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=tensorboard_files

**Reference**


*   Chollet, F. (2021). Deep learning with python. Manning Publications.
*   Geron,Aurelien(2022): Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, O'Reilly Media, Inc. Publications

